# AI - State space search

In [ ]:
# Man, Lion, Goat, Cabbage
# Left bank to Right bank
# Constraint 1 - Lion will eat Goat if man is not there
# Constrain 2 - Goat will eat Cabbage if man is not there
# discrete state space - categorical features Man (left side or right side), similarly other objects

In [ ]:
# wnew = wold - eta * gradL_wold
# state of the network or ml model is indicated by 'w'
# it is changing due to one action
# that action is gradient descent step or weight update step
# continuous state space

In [ ]:
# Actions very specific to the problem domain
# Man goes alone
# Man takes Lion
# Man takes Goat
# Man takes Cabbage

In [ ]:
# The first requirement is state space representation 
# Implicitly or Explicitly

In [ ]:
# Let s[0] - Position of Man
# s[1] - .. Lion
# s[2] .. Goat
# s[3] .. Cabbage
# Positions can be 0 or 1; 0 left side and 1 right side
def penalty(s):
  # constraint 1
  if s[0]!=s[1] and s[1]==s[2]:
    return 1000.0
  # constraint 2
  if s[0]!=s[2] and s[2]==s[3]:
    return 1000.0
  return 0

In [ ]:
def go_alone(s1):
    s2 = s1.copy()
    s2[0] = 1 - s1[0]
    return s2

def take_lion(s1):
  s2 = s1.copy()
  if s2[0]==s2[1]:
    s2[0] = 1 - s2[0]
    s2[1] = 1 - s2[0]
  return s2

def take_goat(s1):
  s2 = s1.copy()
  if s2[0]==s2[2]:
    s2[0] = 1 - s2[0]
    s2[2] = 1 - s2[2]
  return s2

def take_cabbage(s1):
  s2 = s1.copy()
  if s2[0]==s2[3]:
    s2[0] = 1 - s2[0]
    s2[3] = 1 - s2[3]
  return s2


In [ ]:
action_list = [go_alone, take_lion, take_goat, take_cabbage]

In [ ]:
init_state = [0,0,0,0]

In [ ]:
goal_state = [1,1,1,1]

In [ ]:
n_iter = 1000

In [ ]:
def state_copy(s):
  return s.copy()

In [ ]:
tmp_state = state_copy(init_state)

for i in range(n_iter):
  for a in action_list:
    if tmp_state==goal_state:
      print ('Print the solution..')
      break
    tmp_state2 = a(tmp_state)
    if penalty(tmp_state2) == 0 :
      tmp_state = state_copy(tmp_state2)
      # Put some DFS type of logic here
      # Think of some backtracking logic to explore different states

In [ ]:
import torch.nn

In [ ]:
m1 = torch.nn.Softmax(dim=1)
m2 = torch.nn.Softmax(dim=2)

In [ ]:
x = torch.rand(2,3) #its hard to interpret for x = torch.rand(2,3,4)

In [ ]:
print (x)

tensor([[0.2809, 0.2739, 0.4466],
        [0.3563, 0.0080, 0.3194]])


In [ ]:
y1 = m1(x)
print (y1)

tensor([[0.3151, 0.3129, 0.3719],
        [0.3746, 0.2644, 0.3610]])


In [ ]:
y2 = m2(x)
print (y2)

IndexError: ignored

# Symbolic differentiation (own)

In [ ]:
class DerivativeType :
  def __init__(self,name=None):
    self.name = name
  def getname(self):
    return self.name
  def derivative(self,paramlist=None): # backward function of pytorch
    print ('inside derivative',self.name)
    return None
  def compute(self,paramlist=None): # forward function of pytorch
    print ('inside compute',self.name)
    return None
  def __str__(self):
    if self.name is None:
      return 'None'
    return self.name

In [ ]:
def display(derobj_list):
  for der in derobj_list:
    print (der)

In [ ]:
class IllegalDerivativeOperation(Exception):
  def __init__(self):
    pass
  def __str__(self):
    return 'IllegalDerivativeOperation'

x = IllegalDerivativeOperation()

print (x)

IllegalDerivativeOperation


In [ ]:
x = DerivativeType('some name xx')
x.derivative()
x.compute()
print (x)

inside derivative some name xx
inside compute some name xx
some name xx


In [ ]:
class MyConstant(DerivativeType):
  def __init__(self,name,value):
    super().__init__(name=name)
    self.value = value

  def derivative(self,paramlist=None):
    derout_list = []

    if paramlist is not None:
      for param in paramlist:
        z = MyConstant('zero',0)
        derout_list.append(z)
    return derout_list

  def compute(self):
    return self.value

  def __str__(self):
    return '(' + self.name + ')'
    
x = MyConstant('y',3)
print (x.getname())
y = x.derivative([x])
print ('printing derivative...')
display(y)
print (x.compute())

print (x)

y
printing derivative...
(zero)
3
(y)


In [ ]:
try :
  print ('kali before raise')
  raise IllegalDerivativeOperation
  print ('kali after raise')
except Exception as e:
  print (e)

kali before raise
IllegalDerivativeOperation


In [ ]:
class MyVariable(DerivativeType):
  def __init__(self,name, value):
    super().__init__(name)
    self.value = value

  def derivative(self,paramlist=None):
    if paramlist is None:
      raise IllegalDerivativeOperation

    der_list = []
    if paramlist is not None:
      for param in paramlist :
        if param.getname() != self.getname():
          der_list.append(MyConstant('zero',0))
        else :
          der_list.append(MyConstant('one',1))
    return der_list

  def compute(self):
    return self.value

  def __str__(self):
    return '(' + self.name + ')'

x = MyVariable('x',1)
y = MyVariable('y',2)

z = x.derivative([x,y])
display(z)

print (x.compute())

(one)
(zero)
1


In [ ]:
class MyUMinus(DerivativeType):
  def __init__(self,x):
    super().__init__()
    self.x = x
  def derivative(self,paramlist):
    derout_list = []
    if paramlist is not None:
      for param in paramlist:
        z = MyUMinus(self.x.derivative([param])[0])
        derout_list.append(z)
    return derout_list

  def compute(self):
    return -1 * self.x.compute()

  def __str__(self):
    return '-' + str(self.x)

x = MyVariable('x',1)
y = MyVariable('y',23)
ux = MyUMinus(x)

print (ux)

# z = ux.derivative([x,y,x,ux]) ux derivative logic to be fixed
z = ux.derivative([x,y]) 

display(z)

-(x)
-(one)
-(zero)


In [ ]:
class MyAddition(DerivativeType):
  def __init__(self,x,y):
    self.x = x
    self.y = y

  def derivative(self,paramlist=None):
    derout_list = []

    if paramlist is not None:
      for param in paramlist:
        a = self.x.derivative([param])[0]
        b = self.y.derivative([param])[0]
        c = MyAddition(a,b)
        derout_list.append(c)

    return derout_list

  def compute(self):
    z = self.x.compute() + self.y.compute()
    return z

  def __str__(self):
    mystr = '(' + str(self.x) + '+' + str(self.y) + ')'
    return mystr


x = MyVariable('x',1)
y = MyVariable('y',23)

print (x,y)

z = MyAddition(x,y)

print (z)

z1 = z.derivative([x,y])

display(z1)

print (z.compute())

(x) (y)
((x)+(y))
((one)+(zero))
((zero)+(one))
24


In [ ]:
class MyMultiplication(DerivativeType):
  def __init__(self,x,y):
    self.x = x
    self.y = y

  def derivative(self,paramlist=None):
    derout_list = []
    if paramlist is not None:
      for param in paramlist:
        a = MyMultiplication(self.x.derivative([param])[0], y)
        b = MyMultiplication(self.x, self.y.derivative([param])[0])
        c = MyAddition(a,b)
        derout_list.append(c)
    return derout_list

  def compute(self):
    z = self.x.compute() * self.y.compute()
    return z

  def __str__(self):
    return '('+str(self.x) + '*' + str(self.y)+')'
    
  
x = MyVariable('x',1)
y = MyVariable('y',2)

z = MyMultiplication(x,y)

print (z)

t = z.derivative([x,y])

display (t)

((x)*(y))
(((one)*(y))+((x)*(zero)))
(((zero)*(y))+((x)*(one)))


In [ ]:
x = MyVariable('x',1)
y = MyVariable('y',1)
t1 = MyMultiplication(x,y)
t2 = MyAddition(x,t1)
t3 = MyMultiplication(t2,t2)
print (t3)

(((x)+((x)*(y)))*((x)+((x)*(y))))


In [ ]:
z = t3.derivative([x])
display(z)
print (z[0])
print (z[0].compute())

((((one)+(((one)*(y))+((x)*(zero))))*(y))+(((x)+((x)*(y)))*((one)+(((one)*(y))+((x)*(zero))))))
((((one)+(((one)*(y))+((x)*(zero))))*(y))+(((x)+((x)*(y)))*((one)+(((one)*(y))+((x)*(zero))))))
6


In [ ]:
z = t3.derivative([x])

display (z)

((((one)+(((one)*(y))+((x)*(zero))))*(y))+(((x)+((x)*(y)))*((one)+(((one)*(y))+((x)*(zero))))))


In [ ]:
x = MyVariable('x',1)
y = MyVariable('x',1)
t1 = MyAddition(MyMultiplication(x,y),MyConstant('1',1))
print (t1)

dt = t1.derivative([x])

display(dt)

(((x)*(x))+(1))
((((one)*(x))+((x)*(one)))+(zero))


In [ ]:
class MyMulInv(DerivativeType):
  def __init__(self,x):
    super().__init__()
    self.x = x
  def derivative(self,paramlist):
    deroutlist = []
    if paramlist is not None:
      for param in paramlist:
        ...
        ...

In [ ]:
class MyLog(DerivativeType):
  def __init__(self,x):
    super().__init__()
    self.x = x

  def derivative(self,paramlist):
    derout_list = []
    if paramlist is not None:
      for param in paramlist:
        

In [ ]:
class MyExponentiation(DerivativeType):
  def __init__(self,x,y):
    super().__init__()
    self.x = x
    self.y = y

  def derivative(self,paramlist):
    derout_list = []
    if paramlist is not None:
      for param in paramlist:
        a = MyMultiplication(y,MyExponentiation(x,MyAddition(y,MyUMinus(MyConstant('-1')))))
        b = MyMultiplication(MyExponentiation(x,y),MyLog(x))



# Python iterator

# Sympy

# Clarification regarding library function calls

In [ ]:
# the following is an example of a function inside the python library
def myfunction(x):
  x.__myfunction__()

def myfunction2(x):
  x.__myfunction2__()

# end of example library

# your class should implement __xx__() type of function
class A :
  def __init__(self):
    pass
  def __myfunction__(self):
    pass
    print ('inside __myfunction__')

  def __myfunction2__(self):
    print ('inside __myfunction2__')

a = A()

myfunction(a)
myfunction2(a)

inside __myfunction__
inside __myfunction2__


In [ ]:
class MyDataLoader:

  def __init__(self,batch_size):
    self.batch_size = batch_size

  def __iter__(self):
    self.a = 1
    return self

  def __next__(self):
    if self.a <= self.batch_size:
      x = self.a
      self.a += 1
      print ('returning a row',self.a)
      return x
    else:
      raise StopIteration

myclass = MyDataLoader(5)
myiter = iter(myclass)

for x in myiter:
  print(x)

returning a row 2
1
returning a row 3
2
returning a row 4
3
returning a row 5
4
returning a row 6
5


# 'with' clause

In [ ]:
# Exception class
class MyException(Exception):
  def __init__(self,msg):
    self.msg = msg

  def __str__(self):
    mystr = 'My Custom Exception ' + self.msg
    return mystr

In [ ]:
t1 = MyException('abc')
t2 = MyException('pqr')
print (t1)
print (t2)

My Custom Exception abc
My Custom Exception pqr


In [ ]:
try :
  print ('Example try')
  raise MyException('some msg text text')
  print ('After exception is raised (this code will never be reached)')
except Exception as eobj:
  print ('Exception handling code')
  print (eobj)

Example try
Exception handling code
My Custom Exception some msg text text


In [ ]:
class MyClass:
  
  def __init__(self, name):
    print ('initialized',name)
    self.name = name

  def op1(self, params):
    print ('operation 1', params)
  
  def op2(self, params):
    print ('operation 2', params)

  def close(self):
    print ('inside code',self.name)

class MyClassHandler:
    def __init__(self1, name):
        self1.name = name
      
    def __enter__(self1):
        self1.x = MyClass(self1.name)
        return self1.x

    # REF - https://stackoverflow.com/questions/22417323/how-do-enter-and-exit-work-in-python-decorator-classes
    def __exit__(self1,exc_type, exc_val, tb):
      # you can write your own closing logic
      return self1
  


In [ ]:
with MyClassHandler('some name') as xobj:
    xobj.op1('params 1')
    xobj.op2('param 2')
    xobj.op1('param 3')
    xobj.op1('param xx')

initialized some name
operation 1 params 1
operation 2 param 2
operation 1 param 3
operation 1 param xx


# Pytorch - Example Neural Network

In [ ]:
# REF - https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303668  [    0/60000]
loss: 2.295953  [ 6400/60000]
loss: 2.272277  [12800/60000]
loss: 2.268422  [19200/60000]
loss: 2.251653  [25600/60000]
loss: 2.209183  [32000/60000]
loss: 2.222765  [38400/60000]
loss: 2.177465  [44800/60000]
loss: 2.173377  [51200/60000]
loss: 2.145881  [57600/60000]
Test Error: 
 Accuracy: 46.0%, Avg loss: 2.137391 

Epoch 2
-------------------------------
loss: 2.144399  [    0/60000]
loss: 2.136784  [ 6400/60000]
loss: 2.065311  [12800/60000]
loss: 2.091719  [19200/60000]
loss: 2.042312  [25600/60000]
loss: 1.964676  [32000/60000]
loss: 2.009735  [38400/60000]
loss: 1.909845  [44800/60000]
loss: 1.920094  [51200/60000]
loss: 1.860091  [57600/60000]
Test Error: 
 Accuracy: 54.7%, Avg loss: 1.844741 

Epoch 3
-------------------------------
loss: 1.876761  [    0/60000]
loss: 1.847603  [ 6400/60000]
loss: 1.710474  [12800/60000]
loss: 1.770515  [19200/60000]
loss: 1.674961  [25600/60000]
loss: 1.610720  [32000/600

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


# Pytorch - Curve fitting

In [ ]:
# REF - https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

## Numpy based approach

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(0,1,100)
y=3+(4*x)+(5*x**2)+(6*x**3)
#y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-3
for t in range(200):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

99 5.611758628247737
199 3.779481334468664
Result: y = 2.6118888602302226 + 5.455542858858748 x + 5.411139477803916 x^2 + 4.100216399808352 x^3


In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(0,1,1000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()

learning_rate = 1e-3
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    

print(f'Result: y = {a} + {b} x + {c} x^2')

99 8.206591525262678e+51
199 6.163987466120722e+103
299 4.629783432687346e+155
399 3.4774396851712193e+207
499 2.6119119694945756e+259
599 inf
699 inf
799 inf
899 inf
999 inf
1099 inf
1199 nan
1299 nan
1399 nan
1499 nan
1599 nan
1699 nan
1799 nan
1899 nan
1999 nan
Result: y = nan + nan x + nan x^2


/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in square
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars


## Tensor based approach (without autograd)

In [ ]:
# -*- coding: utf-8 -*-

import torch
import math


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(0,1,100, device=device, dtype=dtype)
#y = torch.sin(x)
y=3+(4*x)+(5*x*x)+(6*x*x*x)
# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-3
for t in range(100):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 5.723137855529785
Result: y = 2.678769826889038 + 5.540377616882324 x + 5.480797290802002 x^2 + 3.6540627479553223 x^3


## Tensor based (WITH AUTOGRAD)

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = torch.device("cpu")
#device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(0,1,100, device=device, dtype=dtype)
#y = torch.sin(x)
y=3+(4*x)+(5*x*x)+(6*x*x*x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-3
for t in range(100):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 1.0229239463806152
Result: y = 3.2255990505218506 + 3.4655909538269043 x + 5.047950267791748 x^2 + 6.187089920043945 x^3


## Tensor based WITH AUTOGRAD + OPTIM

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(0,1,100, dtype=dtype)
#y = torch.sin(x)
y=3+(4*x)+(5*x*x)+(6*x*x*x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-3

# 2022 Aug 23 - Kalidas (ykalidas@iittp.ac.in)
optimizer = torch.optim.SGD([a,b,c,d], lr=learning_rate, momentum=0)
    

In [ ]:
for t in range(1000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()

    # 2022 Aug 23 - Kalidas (ykalidas@iittp.ac.in)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if t % 100 == 99:
        print(t, loss.item())

99 0.8174586296081543
199 0.17746180295944214
299 0.13403907418251038
399 0.10160563886165619
499 0.07708349823951721
599 0.058543313294649124
699 0.04452750086784363
799 0.03392999619245529
899 0.02591809630393982
999 0.019859835505485535


In [ ]:
for x in [a,b,c,d]:
  x.requires_grad = False

In [ ]:
print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

Result: y = 2.980103015899658 + 4.01419734954834 x + 5.243313312530518 x^2 + 5.727081298828125 x^3


In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')

99 209.95834350585938
199 144.66018676757812
299 100.70249938964844
399 71.03519439697266
499 50.97850799560547
599 37.403133392333984
699 28.206867218017578
799 21.97318458557129
899 17.7457275390625
999 14.877889633178711
1099 12.93176555633545
1199 11.610918998718262
1299 10.71425724029541
1399 10.10548210144043
1499 9.692106246948242
1599 9.411375045776367
1699 9.220745086669922
1799 9.091285705566406
1899 9.003360748291016
1999 8.943639755249023
Result: y = -5.394172664097141e-09 + -2.208526849746704 * P3(1.367587154632588e-09 + 0.2554861009120941 x)


In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3) 

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 225.2261199951172
199 157.5009765625
299 111.08541870117188
399 79.2381362915039
499 57.36173629760742
599 42.317413330078125
699 31.960010528564453
799 24.821544647216797
899 19.896257400512695
999 16.49434471130371
1099 14.142193794250488
1199 12.514185905456543
1299 11.386262893676758
1399 10.604043960571289
1499 10.061051368713379
1599 9.683794021606445
1699 9.421426773071289
1799 9.238815307617188
1899 9.11160945892334
1999 9.022926330566406
Result: y = -0.013745415024459362 + 0.8508179187774658 x + 0.00237131305038929 x^2 + -0.09248790144920349 x^3


In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use RMSprop; the optim package contains many other
# optimization algorithms. The first argument to the RMSprop constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()


linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 533.353515625
199 383.0157775878906
299 270.6048583984375
399 181.2832489013672
499 114.58033752441406
599 68.02662658691406
699 37.8013916015625
799 20.2047119140625
899 11.908942222595215
999 9.279428482055664
1099 8.891770362854004
1199 8.95720386505127
1299 8.919443130493164
1399 8.907357215881348
1499 8.928251266479492
1599 8.926275253295898
1699 8.918866157531738
1799 8.918336868286133
1899 8.921676635742188
1999 8.9215726852417
Result: y = -0.0005009158630855381 + 0.8562391996383667 x + -0.0005009331507608294 x^2 + -0.09383225440979004 x^3


In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = Polynomial3()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters (defined 
# with torch.nn.Parameter) which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 2885.638916015625
199 1936.072998046875
299 1300.877197265625
399 875.6904296875
499 590.8817138671875
599 399.9662780761719
699 271.89373779296875
799 185.91094970703125
899 128.13821411132812
999 89.28780364990234
1099 63.139102935791016
1199 45.523529052734375
1299 33.645416259765625
1399 25.628427505493164
1499 20.212207794189453
1599 16.54926109313965
1699 14.069504737854004
1799 12.388973236083984
1899 11.248841285705566
1999 10.474541664123535
Result: y = 0.029060767963528633 + 0.827519953250885 x + -0.0050134663470089436 x^2 + -0.08917397260665894 x^3


In [ ]:
# -*- coding: utf-8 -*-
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 1538.4808349609375
3999 696.7999267578125
5999 320.11981201171875
7999 148.0598907470703
9999 72.9036636352539
11999 37.97261047363281
13999 22.187774658203125
15999 14.939539909362793
17999 11.581270217895508
19999 10.067399978637695
21999 9.212265014648438
23999 9.114272117614746
25999 8.727289199829102
27999 8.560704231262207
29999 8.880521774291992
Result: y = 0.005055580288171768 + 0.8542385697364807 x + -0.001442049746401608 x^2 + -0.09324534982442856 x^3 + 0.00011410781007725745 x^4 ? + 0.00011410781007725745 x^5 ?
